In [1]:
# This section introduces pooling layers, 
# which serve the dual purposes of mitigating the sensitivity of convolutional layers to location and of spatially downsampling representations
import torch
from torch import nn
from d2l import torch as d2l

您正确地指出了，汇总信息以生成粗糙图是为使我们的模型逐渐获得对整个图像的全局了解。同时，在处理过程的中间层保留了卷积层的所有优势。在网络中深入的层次越深，与输入相关的隐藏节点的感受野就越大。降低空间分辨率加速了这个过程，因为卷积核覆盖了更大的有效区域。

此外，在检测较低级别的特征（例如边缘，如第 7.2 节所讨论的）时，我们希望生成的表示在某种程度上不受平移影响。例如，如果我们将图像 X （在黑白之间有明显的分界线）整个图像向右平移一个像素，那么新图像 Z 的输出可能会有很大不同。边缘将向右平移一个像素。实际上，在固定的位置上，对象很少出现在完全相同的位置。实际上，即使使用三脚架和静止的物体，由于快门运动引起的相机振动，可能会使整个区域偏移一个像素左右（高端相机装载了专门的功能来解决此问题）。

为了解决平移不变性问题，卷积神经网络（CNN）采用**权值共享**的方式。这样，无论图像特征出现在图像中的何处，卷积层都可以用相同的卷积核检测它们。此外，池化（Pooling）层也有助于在一定程度上实现平移不变性，因为它们可以减少特征图的分辨率，从而使局部变换带来的影响降到最低。通过这两种方法，CNN 在一定程度上可以有效地应对图像中的平移变换。


权值共享是卷积神经网络（CNN）中一个重要的概念。它是指在 CNN 的卷积层中，同一输入通道使用同一个卷积核在输入特征图上滑动进行卷积操作。因此，这个卷积核的权值在整个输入特征图上是共享的。这与在传统的全连接层中，每个输入节点和输出节点之间都有单独的权重相对。

权值共享具有以下优点：

1. **减少参数**：通过在整个输入特征图上共享卷积核权值，我们可以显著减少网络中的参数数量。较少的参数有助于减少内存需求、需要传输的数据，同时能够提高计算效率。

2. **平移不变性**：权值共享提供了网络对输入图像中特征位置变化的平移不变性。因为同一个卷积核在整个图像上滑动，所以无论特征出现在什么位置，都可以被同一个卷积核检测到。这对于识别在图像中以不同位置、大小和方向出现的对象特征非常有帮助。

3. **特征检测**：权值共享允许 CNN 为输入图像中的所有可能位置学习局部特征。这意味着，每个卷积核可以表示为一种特征检测器，其可以在整个图像上执行特定类型的特征提取。

简而言之，权值共享是通过在整个输入特征图上应用相同的卷积核权重来实现的。这降低了计算成本，同时有助于网络实现对输入图像的平移不变性。

In [2]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [4]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

In [5]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [6]:
# Since pooling aggregates information from an area, deep learning frameworks default to matching pooling window sizes and stride
pool2d = nn.MaxPool2d(3)
# Pooling has no model parameters, hence it needs no initialization
pool2d(X)

tensor([[[[10.]]]])

In [7]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [8]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [9]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

池化（Pooling）是一种非常简单的操作。它按照其名称所示，在一定窗口范围内对值进行汇总。所有卷积语义，例如步幅和填充，都以与之前相同的方式应用。注意，池化对通道是不敏感的，即保持通道数量不变，分别对每个通道进行池化。最后，在两种流行的池化选择中，最大池化（Max-Pooling）相对于平均池化（Average Pooling）是优选的，因为它为输出赋予了一定程度的不变性。常见选择是选择大小为 2x2 的池化窗口，将输出的空间分辨率降到原来的四分之一。

需要注意的是，降低空间分辨率的方法远不止池化。例如，在随机池化（Stochastic Pooling）（Zeiler 和 Fergus，2013）和分数最大池化（Fractional Max-Pooling）（Graham，2014）中，聚合过程与随机过程相结合。在某些情况下，这可以稍微提高精度。最后，正如我们稍后将在注意力机制中看到的那样，在聚合输出方面有更精细的方法，例如使用查询和表示向量之间的对齐。

1. 通过卷积实现平均池化：

平均池化可以通过卷积操作实现。将卷积核的所有元素设为 1，并除以它们的总和。例：对于 2x2 的平均池化，卷积核如下：

```
1/4 1/4
1/4 1/4
```

应用这个卷积核，并使步长等于池化窗口大小即可实现平均池化。

2. 证明仅通过卷积操作无法实现最大池化：

卷积操作被设计用来执行在局部区域上的线性组合。而最大池化需要选取区域中的最大值，它是一种非线性操作。这意味着不能仅通过卷积操作实现最大池化，因此需要单独的最大池化方法来处理这种非线性。

3. 使用 ReLU 操作实现最大池化，即 ReLU(x)=max(0, x)。

3.1 使用仅 ReLU 操作表示 max(a, b)。

max(a, b) 可由以下 ReLU 表达式表示：

```
max(a, b) = a + b - ReLU(a - b)
```

3.2 使用卷积和 ReLU 层实现最大池化：

最大池化无法直接通过卷积层和 ReLU 层实现，因此第 3.2 题无法回答。

3.3 对于 2x2 卷积，需要多少个通道和层？对于 3x3 卷积，需要多少？

这个问题提及的上下文不明确。在卷积神经网络中，通道数和层数取决于特定任务和设计。此外，问题与最大池化和 ReLU 不相关。

4. 池化层的计算成本是多少？假设输入到池化层的大小为 c×h×w，池化窗口具有 p_h×p_w 的形状，填充为 (p_h, p_w) 并且步长为 (s_h, s_w)。

池化层的计算成本：

```
c * ((h - p_h + 2*p_h) / s_h + 1) * ((w - p_w + 2*p_w) / s_w + 1) * p_h * p_w
```

5. 为什么预期最大池化和平均池化操作方式不同？

最大池化和平均池化作用不同，因为它们在汇聚局部特征时采用了不同的操作。最大池化旨在选取局部区域内的最大值，而平均池化则对局部区域内的所有值求均值。这两种操作在特征学习方面的差异可以简化为：

   - 最大池化：突出局部特征中最显著的特征（噪声抑制，保留感兴趣区域）。
   
   - 平均池化：各种特征的均匀融合（最终结果可能受噪声等因素影响）。

6. 我们需要单独的最小池化层吗？能否用其他操作代替？

可以将最小池化层替换为其他操作，例如输入数据的取反（乘以 -1），然后应用最大池化，再对结果再取反。

7. 我们可以使用 softmax 操作进行池化。为什么它可能不那么流行？

尽管使用 softmax 操作进行池化在某些情况下可能有效，为什么它可能不那么流行，原因如下：

1. **计算成本**：softmax 运算在池化操作中引入了额外的计算成本。相较于最大池化和平均池化，它需要更多的计算和归一化。这使得 softmax 操作成为一个不太经济的选择。

2. **缺乏明确保留特征**：在最大池化中，局部区域的最显著信息得以保留。然而，在 softmax 池化中，所有输入都被归一化，导致区分度变得模糊，这使得它在某些情况下的表示能力不如最大池化。

3. **梯度问题**：softmax 函数在数值相差较大的情况下可能导致梯度消失问题。在反向传播过程中，这会影响网络的学习效果。

虽然 softmax 操作在某些特定任务和网络结构中可能具有一定优势，但由于以上原因，它作为一种池化策略在实际应用中并不常见。最大池化和平均池化操作由于它们的简单性、计算效率和特征保留能力，仍然是目前广泛使用的池化方法。